# Generate Video From Data

In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm

# input setting
root_path  = r'output\huawei_gutter\post_data\pinhole'
frames = 50
cam = 'ph_rgb8'

# video setting
out_video_name = f'huawei_gutter_{cam}.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 25 # frames/s
out_size = (480, 480)
is_color = True

video_writer = cv2.VideoWriter(out_video_name, fourcc, fps, out_size, is_color)

for i in tqdm(range(frames)):
    img = cv2.imread(os.path.join(root_path, f'{cam}_{i}.jpg'))
    img = cv2.resize(img, out_size)
    video_writer.write(img)

video_writer.release()



# Pinehole Data Visualization

In [9]:
import cv2
import numpy as np
import os
import cv2

cam = 'ph_rgbL'
scene = 'robot_vaccum_scene3'
root_path = rf'output\{scene}\raw_data\{cam}'
frames = 1920


if __name__ == '__main__':
    try:
        for i in range(frames):
            raw_data = np.load(os.path.join(root_path,f'{cam}_{i}.npz'))['data']
            raw_data = cv2.resize(raw_data, (640, 480))
            cv2.imshow('img', raw_data)
            cv2.waitKey(-1)
    finally:
        cv2.destroyAllWindows()

# Change Weather

In [ ]:
import carla

client = carla.Client('127.0.0.1', 2000)
world = client.get_world()
weather = world.get_weather()

weather.cloudiness = 20
weather.wetness = 100
weather.precipitation = 50
# weather.precipitation_deposits = 0
# weather.fog_falloff = 0.8
# weather.fog_density=0
# weather.sun_azimuth_angle = -45
# weather.sun_altitude_angle = 30
# weather.dust_storm = 0
world.set_weather(weather)

print(weather.wetness)


# Check Control File

In [1]:
import numpy as np

params_vc = np.load(r'D:\heterogenenous-data-carla-main\output\huawei_fulldepth\configs\drive_files\68_transforms.npy')

print('Frames: {}'.format(len(params_vc)))

Frames: 129


# Vis ERP Depth

In [8]:
import torch.nn as nn
import torch

a = torch.Tensor(4,32)
eb = nn.Embedding(4,32)
b = a[:,None,None,:]


Embedding(4, 32) torch.Size([4, 32])


# make spectator ctrl file

In [1]:
import numpy as np
import re

frames = 1000

with open(r'temp.txt', 'r') as f:
    pattern = r"Translation: X=([\d.-]+) Y=([\d.-]+) Z=([\d.-]+) Rotation: P=([\d.-]+) Y=([\d.-]+) R=([\d.-]+)"
    lines = f.readlines()
    res = []
    random_rot = np.random.randn(frames, 3).astype(np.float32)*5
    random_pos = np.random.rand(frames, 3).astype(np.float32)
    
    random_rot[:,0]=0.0
    random_rot[:,2]=0.0
    
    random_pos[:,0]=(random_pos[:,0]*2-1)*0.05
    random_pos[:,1]=(random_pos[:,1]*2-1)*0.05
    random_pos[:,2]=random_pos[:,2]*0.04
    
    
    random_idx = np.random.randint(0, len(lines), (frames,1))
    print(random_rot.shape, random_idx.shape)
    for line in lines:
        # print(line[0:-1])
        matches = re.search(pattern, line)
        if matches:
            x = matches.group(1)
            y = matches.group(2)
            z = matches.group(3)
            pitch = matches.group(4)
            yaw = matches.group(5)
            roll = matches.group(6)
            item = np.array([x, y, z, pitch, yaw, roll], dtype=np.float32).reshape(1, -1)
            res.append(item)
    res = np.concatenate(res, axis=0)
    np.savez('random1.npz', random_rot=random_rot, random_pos=random_pos, random_idx=random_idx, data=res)

(1000, 3) (1000, 1)


In [18]:
import numpy as np

bag = np.load('random1.npz')
idx = bag['random_idx']
rot = bag['random_rot']
data = bag['data']
print(idx.shape, idx.dtype, idx[0][0].dtype)
print(rot.shape, rot.dtype, rot[0].shape)
print(data.shape, data.dtype)

(1024, 1) int32 int32
(1024, 3) float32
(8, 6) float32
